# Predict female walking speed removing heterogeneity in either integration or adaptation time constants

In [1]:
%matplotlib inline
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy import signal, stats
from sklearn import linear_model
import sys
import warnings; warnings.filterwarnings("ignore")

from aux import get_seg, rand_string
from disp import set_plot
from my_torch import skl_fit_ridge
from my_stats import nanpearsonr

from record_0_main import smlt_ma

cc = np.concatenate

FPS = 30.03  # sampling rate of behavioral data
DT = 1/FPS

STRAINS = ['NM91', 'ZH23']
STRAIN_KEY = '_'.join(STRAINS).lower()

PTRAIN = .8
NSPLIT = 30

TARG_BHV = 'MTN'
TWDWS = [.03, 1, 60]
TARGS = [f'{TARG_BHV}_MN_{twdw}' for twdw in TWDWS]

ALPHA = 10

In [2]:
PFX_BHV = f'data/simple/behav_xtd/behav_xtd'
PFX_NRL = f'data/simple/mlv/ma_vary_het/temp_nrl/temp_ma_vary_het_{rand_string(8)}'

MASK_PFX = f'data/simple/masks/mask'

FSTRAIN = 'data/simple/strains.csv'
MSTRAINS = [(pd.read_csv(FSTRAIN)['STRAIN'] == strain) for strain in STRAINS]
MSTRAIN = np.any(MSTRAINS, axis=0)
ISTRAIN = MSTRAIN.nonzero()[0]

NTR = len(ISTRAIN)
NTRAIN = int(round(PTRAIN*NTR))

Load behavioral data from Coen et al 2014

In [3]:
df_behav = pd.read_csv('data/simple/c_song_f_behav.csv')
df_behav

,ID,FRAME,T,Q,S,P,F,FFV,FFA,FLS,FRS,FMFV,FMLS
0,0,0,0.000000,1,0,0,0,0.231,0.0000,0.159,0.077,-0.058526,0.274257
1,0,1,0.033300,1,0,0,0,0.314,0.0385,0.025,0.044,0.100623,0.298490
2,0,2,0.066600,1,0,0,0,0.308,0.0025,0.029,0.011,0.147629,0.271865
3,0,3,0.099900,1,0,0,0,0.319,0.0020,0.016,0.016,0.091174,0.306112
4,0,4,0.133200,1,0,0,0,0.312,-0.0315,0.030,0.002,0.065871,0.306439
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4978565,275,27100,902.430902,1,0,0,0,0.217,0.0280,0.029,0.058,-0.171418,0.136184
4978566,275,27101,902.464202,1,0,0,0,0.210,-0.1045,0.138,0.322,0.032522,0.249171
4978567,275,27102,902.497502,1,0,0,0,0.008,-0.0910,0.034,0.260,0.033202,0.010844
4978568,275,27103,902.530803,1,0,0,0,0.028,0.0695,0.010,0.090,0.012059,0.027177


In [4]:
# split big df into dfs for individual trials
n_tr = np.max(df_behav.ID) + 1
dfs_tr = [df_behav[df_behav.ID == i] for i in ISTRAIN]

df_behav = None

In [5]:
# function for recording surrogate neural activity and predicting behavior
def record_fit(tau_rs, tau_as, x_ss, x_ps):
    n = len(tau_rs)
    r_cols = [f'R_{cr}' for cr in range(n)]
    
    # record data
    sys.stdout.write('R')
    for itr, df_tr in zip(ISTRAIN, dfs_tr):
        df_tr_nrl = df_tr.copy()

        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)

        # simulate population response
        params = {'TAU_R': tau_rs, 'TAU_A': tau_as, 'X_S': x_ss, 'X_P': x_ps}
        rs = smlt_ma(i_s, i_p, params, DT)

        df_tr_nrl[r_cols] = rs

        np.save(f'{PFX_NRL}_tr_{itr}.npy', np.array([{'df': df_tr_nrl}]))
        
    sys.stdout.write('F')
    # fit neural -> behav regression model
    rslts = skl_fit_ridge(
        pfxs=[PFX_NRL, PFX_BHV],
        cols_x=r_cols,
        targs=TARGS,
        itr_all=ISTRAIN,
        ntrain=NTRAIN,
        nsplit=NSPLIT,
        alpha=ALPHA,
        mask_pfx=MASK_PFX,
        verbose='dots')
    
    r2_trains = {targ: np.array([rslt.r2_train[targ] for rslt in rslts]) for targ in TARGS}
    r2_tests = {targ: np.array([rslt.r2_test[targ] for rslt in rslts]) for targ in TARGS}
    
    return r2_trains, r2_tests

# Sweep over fixed values of adaptation time constant

In [6]:
N = 20

r2_trains = []
r2_tests = []

rtau_r = [20, 120]
TAU_AS = [.1, .2, .5, 1, 2, 5]

r2_trains_all = {targ: [] for targ in TARGS}
r2_tests_all = {targ: [] for targ in TARGS}

for tau_a in TAU_AS:
    sys.stdout.write(f'tau_a = {tau_a}')
    
    r2_trains = {targ: [] for targ in TARGS}
    r2_tests = {targ: [] for targ in TARGS}
    
    np.random.seed(None)
    tau_rs = np.random.uniform(*rtau_r, N)
    tau_as = np.repeat(tau_a, N)
    x_ss = np.random.uniform(0, 1, N)
    x_ps = 1 - x_ss

    r2_train_mns, r2_test_mns = record_fit(tau_rs, tau_as, x_ss, x_ps)[:2]

    # save training and test r2s and corresponding params
    np.save(f'data/simple/mlv/ma_vary_het/ma_slow_int_fast_adapt_tau_a_{tau_a}.npy', np.array([{
        'R2_TRAIN': r2_train_mns, 'R2_TEST': r2_test_mns, 'TAU_R': tau_rs, 'TAU_A': tau_a, 'X_S': x_ss, 'X_PS': x_ps,
    }]))
    
    print('')

tau_a = 0.1RF..............................
tau_a = 0.2RF..............................
tau_a = 0.5RF..............................
tau_a = 1RF..............................
tau_a = 2RF..............................
tau_a = 5RF..............................


# Sweep over fixed values of integration time constant

In [ ]:
N = 20

r2_trains = []
r2_tests = []

TAU_RS = [20, 30, 60, 90, 120, 240]
rtau_a = [.1, 2]

r2_trains_all = {targ: [] for targ in TARGS}
r2_tests_all = {targ: [] for targ in TARGS}

for tau_r in TAU_RS:
    sys.stdout.write(f'tau_r = {tau_r}')
    
    r2_trains = {targ: [] for targ in TARGS}
    r2_tests = {targ: [] for targ in TARGS}
    
    np.random.seed(None)
    tau_rs = np.repeat(tau_r, N)
    tau_as = np.random.uniform(*rtau_a, N)
    x_ss = np.random.uniform(0, 1, N)
    x_ps = 1 - x_ss

    r2_train_mns, r2_test_mns = record_fit(tau_rs, tau_as, x_ss, x_ps)[:2]

    # save training and test r2s and corresponding params
    np.save(f'data/simple/mlv/ma_vary_het/ma_slow_int_fast_adapt_tau_r_{tau_r}.npy', np.array([{
        'R2_TRAIN': r2_train_mns, 'R2_TEST': r2_test_mns, 'TAU_R': tau_r, 'TAU_A': tau_as, 'X_S': x_ss, 'X_PS': x_ps,
    }]))
    
    print('')